In [6]:
import pyspark
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [62]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2023-03-03 16:11:43--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.131.121, 54.231.140.201, 54.231.196.177, ...
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.131.121|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2023-03-03 16:11:44 ERROR 403: Forbidden.



11908469 fhvhv_tripdata_2021-01.csv


In [11]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-06.csv')

In [12]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [13]:
!head -n 1001 fhvhv_tripdata_2021-06.csv > head1.csv

In [14]:
import pandas as pd

In [15]:
df_pandas = pd.read_csv('head1.csv')

In [16]:
df_pandas.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [17]:
spark.createDataFrame(df_pandas).schema

TypeError: field Affiliated_base_number: Can not merge type <class 'pyspark.sql.types.StringType'> and <class 'pyspark.sql.types.DoubleType'>

Integer - 4 bytes
Long - 8 bytes

In [18]:
from pyspark.sql import types

In [19]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)]
)

In [34]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv')

AnalysisException: Path does not exist: file:/Users/kudriavtcevi/Desktop/data-zoomcamp-2023/data-zoomcamp-2023/spark_batch_processing/homework/fhvhv_tripdata_2021-06.csv

In [31]:
df = df.repartition(12)

In [32]:
df.write.parquet('fhvhv/2021/06/')

23/03/03 15:05:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [34]:
df = spark.read.parquet('fhvhv/2021/06/')

In [21]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [22]:
from pyspark.sql import functions as F

In [23]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
|              B02510|2021-06-01 00:18:1

In [24]:
df.createOrReplaceTempView('trips_data')


In [25]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [41]:
spark.sql("""
SELECT
    count(1)
FROM
    trips_data
WHERE CAST(pickup_datetime as DATE) = CAST('2021-06-15' AS DATE)
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [48]:
spark.sql("""
SELECT
    PULocationID, count(1)
FROM    trips_data
GROUP BY 1
ORDER BY 2 DESC
LIMIT 100
""").show()

+------------+--------+
|PULocationID|count(1)|
+------------+--------+
|          61|  231279|
|          79|  221244|
|         132|  188867|
|          37|  187929|
|          76|  186780|
|         231|  164344|
|         138|  161596|
|         234|  158937|
|         249|  154698|
|           7|  152493|
|         148|  151020|
|          68|  147673|
|          42|  146402|
|         255|  143683|
|         181|  143594|
|         225|  141427|
|          48|  139611|
|         246|  139431|
|          17|  138428|
|         170|  137879|
+------------+--------+
only showing top 20 rows



In [33]:
spark.sql("""
SELECT
    bigint(dropoff_datetime)- bigint(pickup_datetime), (bigint(dropoff_datetime)- bigint(pickup_datetime))/3600.0
FROM
    trips_data
ORDER BY 1 DESC
LIMIT 100
""").show()

23/03/03 16:44:09 ERROR Executor: Exception in task 6.0 in stage 6.0 (TID 18)
java.io.FileNotFoundException: 
File file:/Users/kudriavtcevi/Desktop/data-zoomcamp-2023/data-zoomcamp-2023/spark_batch_processing/homework/fhvhv_tripdata_2021-06.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:661)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:270)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.has

Py4JJavaError: An error occurred while calling o114.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 6.0 failed 1 times, most recent failure: Lost task 5.0 in stage 6.0 (TID 17) (kudriavcevisair executor driver): java.io.FileNotFoundException: 
File file:/Users/kudriavtcevi/Desktop/data-zoomcamp-2023/data-zoomcamp-2023/spark_batch_processing/homework/fhvhv_tripdata_2021-06.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:661)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$

a:212)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:270)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.convert.Wrappers$IteratorWrapper.hasNext(Wrappers.scala:32)
	at org.sparkproject.guava.collect.Ordering.leastOf(Ordering.java:628)
	at org.apache.spark.util.collection.Utils$.takeOrdered(Utils.scala:37)
	at org.apache.spark.sql.execution.TakeOrderedAndProjectExec.$anonfun$doExecute$4(limit.scala:231)
	at org.apache

In [26]:
schema_zones = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True), 
    types.StructField('Borough', types.StringType(), True), 
    types.StructField('Zone', types.StringType(), True), 
    types.StructField('service_zone', types.StringType(), True)
    ])

In [27]:
df_zones = spark.read \
    .option("header", "true") \
    .schema(schema_zones) \
    .csv('taxi+_zone_lookup.csv')

In [28]:
df_zones.schema

StructType([StructField('LocationID', IntegerType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [30]:
df_result = df.join(df_zones, df.PULocationID == df_zones.LocationID)

In [31]:
df_result.createOrReplaceTempView('trips_data_res')

In [32]:
spark.sql("""
SELECT
    PULocationID, zone, count(1)
FROM    trips_data_res
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 100
""").show()

+------------+--------------------+--------+
|PULocationID|                zone|count(1)|
+------------+--------------------+--------+
|          61| Crown Heights North|  231279|
|          79|        East Village|  221244|
|         132|         JFK Airport|  188867|
|          37|      Bushwick South|  187929|
|          76|       East New York|  186780|
|         231|TriBeCa/Civic Center|  164344|
|         138|   LaGuardia Airport|  161596|
|         234|            Union Sq|  158937|
|         249|        West Village|  154698|
|           7|             Astoria|  152493|
|         148|     Lower East Side|  151020|
|          68|        East Chelsea|  147673|
|          42|Central Harlem North|  146402|
|         255|Williamsburg (Nor...|  143683|
|         181|          Park Slope|  143594|
|         225|  Stuyvesant Heights|  141427|
|          48|        Clinton East|  139611|
|         246|West Chelsea/Huds...|  139431|
|          17|             Bedford|  138428|
|         